# RAG(검색 증강 생성)를 사용한 자연어를 SQL로 변환하기
RAG를 활용하여 자연어를 SQL로 변환하는 성능을 향상시키는 방법

---
## 권장 SageMaker 환경

SageMaker 이미지: sagemaker-distribution-cpu

커널: Python 3

인스턴스 타입: ml.m5.large

---
## 목차

1. [필요한 패키지 설치](#1단계-필요한-패키지-설치)
1. [Bedrock 임베딩 모델 구성](#2단계-bedrock-임베딩-모델과-llm-구성)
1. [Athena와 Bedrock 클라이언트 구성](#3단계-athena와-bedrock-클라이언트-구성)
1. [도우미 함수 생성](#4단계-도우미-함수-생성)
1. [Bedrock 임베딩 모델 구성](#5단계-bedrock-임베딩-모델-구성)
1. [TPC-DS 메타데이터 가져오기](#6단계-tpc-ds-데이터셋-테이블과-컬럼-정보-가져오기)
1. [질문과 메타데이터 임베딩](#7단계-모든-질문과-메타데이터-임베딩하기)
1. [프롬프트 구성 및 쿼리 생성](#8단계-프롬프트-구성하여-sql-쿼리-생성하기)

---

## 목표
이 노트북은 자연어 질문을 해당 질문에 답하는 SQL 쿼리로 변환하는 방법을 구현하는 데 도움이 되는 코드 예제를 제공합니다.

---
## 자연어-SQL 변환 문제에 대한 접근 방식

Bedrock 임베딩(embedding) 모델과 LLM(대형 언어 모델)을 설정하여 테이블 메타데이터를 임베딩하는 과정을 살펴보겠습니다.

**RAG(검색 증강 생성)란?**
RAG는 기존 지식 베이스에서 관련 정보를 검색하여 AI 모델의 답변을 향상시키는 기술입니다. 이를 통해 더 정확하고 맥락에 맞는 답변을 생성할 수 있습니다.

**1단계: Athena에서 메타데이터 가져오기**
먼저 Athena(AWS의 쿼리 서비스)에서 데이터베이스의 메타데이터(테이블 구조, 컬럼 정보 등)를 가져옵니다.

**2단계: 질문 생성**
메타데이터를 활용하여 LLM에게 각 테이블로 답할 수 있는 가능한 질문들을 생성하도록 요청합니다.

**3단계: 벡터 저장소에 임베딩**
모든 메타데이터와 생성된 질문들을 벡터 저장소에 임베딩합니다. 
- **임베딩(Embedding)**: 텍스트를 숫자 벡터로 변환하여 컴퓨터가 의미를 이해할 수 있게 하는 기술
- **FAISS**: 페이스북에서 개발한 빠른 벡터 검색 라이브러리
- **의미적 유사성(Semantic Similarity)**: 단어의 의미적 관련성을 측정하는 방법

**4단계: 프롬프트 설계**
마지막으로 임베딩, 지시사항, 몇 개의 예시(few-shot examples), 그리고 물론 우리의 질문을 포함하는 강력한 프롬프트를 설계합니다.

![Alt text](./content/rag.png)

### 사용 도구
Langchain, Amazon Bedrock SDK (boto3)

---
### 1단계: 필요한 패키지 설치

이 노트북을 실행하는 데 필요한 모든 패키지를 설치합니다.

In [ ]:
!python -m ensurepip --upgrade
%pip install -qU sqlalchemy
%pip install -q "boto3~=1.34" 
%pip install -qU jinja2
%pip install -qU botocore
%pip install -qU pandas
%pip install -qU PyAthena
%pip install -qU faiss-cpu
%pip install -qU langchain
%pip install -qU langchain-aws
%pip install -qU jq

---
### 2단계: Bedrock 임베딩 모델과 LLM 구성

In [ ]:
import os
import sys
import json
from functools import partial
import json
import re

import boto3
from botocore.config import Config
from langchain.document_loaders.json_loader import JSONLoader
from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain_aws import BedrockEmbeddings
from functools import reduce
from langchain.prompts import PromptTemplate
from sqlalchemy import MetaData
from sqlalchemy import create_engine

sys.path.append('../')
from libs.din_sql import din_sql_lib as dsl
import utilities as u

---
### 3단계: Athena와 Bedrock 클라이언트 구성

In [ ]:
ATHENA_RESULTS_S3_LOCATION, ATHENA_CATALOG_NAME = \
    u.extract_CF_outputs("AthenaResultsS3Location", "AthenaCatalogName")
DB_NAME = "tpcds1"
DB_FAISS_PATH = './vectorstore/db_faiss'

ATHENA_RESULTS_S3_LOCATION, ATHENA_CATALOG_NAME, DB_NAME, DB_FAISS_PATH

In [ ]:
model_id: str = "anthropic.claude-v2"
# model_id: str = "amazon.titan-tg1-large"
temperature: float = 0.2
top_k: int = 200

In [ ]:
bedrock_region = athena_region = boto3.session.Session().region_name

In [ ]:
retry_config = Config(retries={'max_attempts': 100})
session = boto3.Session(region_name=bedrock_region)
bedrock = session.client('bedrock-runtime', region_name=bedrock_region,
                         config=retry_config)

---
### 4단계: 도우미 함수 생성

In [ ]:
run_bedrock = partial(u.run_bedrock_simple_prompt,
                      system_prompts=[],
                      model_id=model_id,
                      temperature=temperature,
                      top_k=top_k)

LLM이 주어진 테이블과 컬럼으로 답할 수 있는 질문 목록을 반환하면, `write_questions_to_file` 메서드가 이를 로컬에 json 파일로 저장하는 역할을 합니다.

In [ ]:
def write_questions_to_file(question_list_filename: str,
                            table_name: str,
                            table_schema, answer):
    data_list = []
    question_list_obj = answer
    questions_list = question_list_obj.splitlines()
    print(questions_list)
    # Open the file in write mode
    with open(question_list_filename, mode="w", newline="") as file:
        for question in questions_list:

            # Skip if it doesn't really have a question
            if "?" not in question:
                continue

            questionSplit = re.split(r"\d{1,5}.||. ||- ", question, maxsplit=1)
            print(questionSplit)
            question = questionSplit[1]
            data = {
                "tableName": table_name,
                "question": question,
                "tableSchema": table_schema.lstrip(" "),
            }
            data_list.append(data)

        json.dump(data_list, file)

문서 목록을 받아서 메타데이터가 첨부된 동일한 문서 목록을 반환하는 메서드가 필요합니다. 또한 JSON을 로드하여 JSON 객체를 반환하는 도우미 함수도 필요합니다.

In [ ]:
def create_docs_with_correct_metadata(documents):
    """
    인덱싱에 필요한 올바른 메타데이터를 가진 새로운 문서 생성
    """
    # 새로운 문서 목록을 반환할 예정
    new_docs = []

    # 각 문서에 대해
    for doc in documents:
        # 메타데이터와 내용 가져오기
        metadata = doc.metadata
        contents = json.loads(doc.page_content)

        # 추가하고자 하는 새로운 메타데이터 계산
        new_metadata = {
            "tableName": contents["tableName"],
            "question": contents["question"],
            "tableSchema": contents["tableSchema"],
        }

        # 문서의 새로운 메타데이터 출력
        # print(new_metadata)

        new_docs.append(
            Document(page_content=new_metadata["question"], metadata=new_metadata)
        )

    return new_docs

def load_json_file(filename):
    loader = JSONLoader(file_path=filename, jq_schema=".[]", text_content=False)

    # 이것은 내부 Langchain 문서 데이터 구조입니다
    docs = loader.load()
    return docs

이 함수는 LLM에게 테이블 스키마를 검사하고, 해당 스키마로 답할 수 있는 질문들을 생성하도록 요청한 다음, 이러한 질문들을 파일에 저장합니다. 마지막으로 모든 질문을 단일 벡터 데이터베이스에 추가합니다.
아래 프롬프트는 자연어로 질문을 생성하는 데 사용되며, 질문과 테이블 메타데이터를 임베딩하는 모든 도우미 함수를 호출합니다.

In [ ]:
def add_new_table(schema, table_name,model_id,is_incremental, bedrock_embeddings):
    """
    LLM에게 테이블 스키마를 검사하고, 해당 스키마로 답할 수 있는 질문들을 생성하도록 요청한 다음,
    이러한 질문들을 파일에 저장하고, 모든 질문을 단일 벡터 DB에 로드합니다.

    :schema         : 테이블 스키마
    :table_name     : 테이블 이름
    :model_id       : 모델 ID
    :is_incremental : 증분 처리 여부
    """
    print(f"테이블 {table_name}을 스키마 {schema}와 함께 추가 중")
    prompt = f"""
    {table_name} 테이블의 다음 스키마로 답할 수 있는 고유하고 상세한 질문들의 번호가 매겨진 목록만 반환하세요:
    {schema}.
    지시사항:
        자연어 설명만 사용하세요.
        SQL을 사용하지 마세요.
        다양한 질문 목록을 생성하되, 질문들은 고유하고 상세해야 합니다.
        질문들은 이해하고 답하기 쉬운 형식이어야 합니다.
        테이블의 정보에 대해 가능한 많이 질문하세요.
        한 번에 데이터의 여러 측면에 대해 질문할 수 있습니다.
        질문은 '무엇을', '어떤', '어떻게', '언제' 또는 '할 수 있는가'로 시작해야 합니다. 변수 이름을 사용하세요.
        질문들은 관련된 비즈니스 어휘와 용어만 사용해야 합니다.
        출력에 컬럼명을 사용하지 마세요 - 관련된 자연어 설명만 사용하세요.
        숫자 값을 출력하지 마세요.
        번호가 매겨진 목록으로 시작하는 질문들을 출력하세요.

        \n 질문들: 1.
        """
    answer = run_bedrock(prompt=prompt)
    question_list_filename = f"../questionList{table_name}.json"
    print(f"질문을 {question_list_filename}에 저장 중, 스키마 {schema}, "
          f"테이블 이름 {table_name}, 답변 {answer}.\n\n")
    write_questions_to_file(question_list_filename, table_name, schema, answer)
    docs = load_json_file(question_list_filename)
    docs = create_docs_with_correct_metadata(docs)
    print(f"문서들:\n{docs}")
    new_questions = FAISS.from_documents(docs, bedrock_embeddings)
    db_exists = True if os.path.exists(f"{DB_FAISS_PATH}/index.faiss") else False
    # 새 테이블 추가
    if is_incremental and db_exists:
            question_db = FAISS.load_local(DB_FAISS_PATH, bedrock_embeddings,
                                           allow_dangerous_deserialization=True)
            question_db.merge_from(new_questions)
            question_db.save_local(DB_FAISS_PATH)
    # 처음 로드
    else:
        print(f"is_incremental이 {str(is_incremental)}로 설정되었고/또는 벡터 DB를 찾을 수 없습니다. 생성 중...")
        new_questions.save_local(DB_FAISS_PATH)

 ---
 ### 5단계: Bedrock 임베딩 모델 구성
 여기서는 텍스트를 벡터 임베딩으로 변환하는 LangChain 임베딩 모델을 생성합니다.
 
 **임베딩 모델이란?**
 텍스트를 컴퓨터가 이해할 수 있는 숫자 벡터로 변환하는 AI 모델입니다. 이를 통해 텍스트 간의 의미적 유사성을 계산할 수 있습니다.

In [ ]:
bedrock_embeddings = BedrockEmbeddings(client=bedrock)

 ---
 ### 6단계: TPC-DS 데이터셋 테이블과 컬럼 정보 가져오기
 
 **TPC-DS란?**
 TPC-DS는 데이터 웨어하우스 성능을 측정하기 위한 표준 벤치마크 데이터셋입니다. 실제 비즈니스 시나리오를 모방한 다양한 테이블과 데이터를 포함합니다.

In [ ]:
def get_sqlalchemy_athena(database, catalog, s3stagingathena, region):
    athena_connection_str = f'awsathena+rest://:@athena.{region}.amazonaws.com:443/{database}?s3_staging_dir={s3stagingathena}&catalog_name={catalog}'
    # Athena 엔진 생성
    return create_engine(athena_connection_str)


def get_tpc_ds_dataset(database, catalog, s3stagingathena, region):
    """ 데이터베이스 스키마 반영 """

    column_table  = []
    columns_str = ''
    table_name = ''
    metadata = MetaData()
    engine = get_sqlalchemy_athena(database, catalog, s3stagingathena, region)
    metadata.reflect(bind=engine)

    # 테이블 이름 목록 가져오기
    print(metadata.tables.keys()) 

    # 테이블별로 반복
    for table in metadata.tables:
        print(f"테이블: {table}")
        table_name = table
        columns_str = ""
        print(f"스키마: {metadata.tables[table].schema}")
        print(f"컬럼들: {metadata.tables[table].columns.keys()}")
        for column in metadata.tables[table].columns.keys():
            columns_str = columns_str + f"{column}" + "|"
        column_table.append((columns_str, table_name))
    return column_table

In [ ]:
tpc_ds = get_tpc_ds_dataset(DB_NAME, ATHENA_CATALOG_NAME,
                            ATHENA_RESULTS_S3_LOCATION, athena_region)

---
### 7단계: 모든 질문과 메타데이터 임베딩하기
여기서는 도우미 함수를 사용하여 테이블 메타데이터를 임베딩하고 테이블에 대해 질문할 수 있는 가능한 질문들을 생성합니다.

**이 과정에서 일어나는 일:**
1. 각 테이블의 스키마를 분석
2. AI가 해당 테이블로 답할 수 있는 자연어 질문들을 생성
3. 질문들을 벡터로 변환하여 저장
4. 나중에 사용자 질문과 유사한 질문을 빠르게 찾을 수 있도록 준비

**다음 셀은 일반적으로 실행하는 데 약 5분 정도 걸립니다.**

In [ ]:
for x in tpc_ds:
    print(x)
    schema, table_name = x
    add_new_table(
        schema=schema,
        table_name=table_name,
        model_id=model_id,
        is_incremental=True,
        bedrock_embeddings=bedrock_embeddings)
print("\n-----------------\nFinished embedding metadata")

In [ ]:
question_db = FAISS.load_local(DB_FAISS_PATH, bedrock_embeddings,
                               allow_dangerous_deserialization=True)

In [ ]:
query = "Find the top 10 customer name by total dollars spent"

---
### 8단계: 프롬프트 구성하여 SQL 쿼리 생성하기
먼저 유사성 검색과 키워드 검색을 모두 사용하여 질문의 의미적 의미를 바탕으로 가능한 일치 항목을 찾아 테이블과 컬럼 정보를 가져옵니다.

**이 단계에서 하는 일:**
1. 사용자의 자연어 질문을 분석
2. 벡터 저장소에서 관련 테이블과 컬럼 정보 검색
3. 검색된 정보를 바탕으로 SQL 쿼리 생성을 위한 프롬프트 구성

In [ ]:
schema = {}
results_with_scores = question_db.similarity_search_with_score(query)
for doc, score in results_with_scores:
    print(doc.metadata['question'])
    schema[doc.metadata['tableName']] = doc.metadata['tableSchema']

Anthropic Claude v2 모델로 DIN_SQL 클래스를 초기화합니다.

In [ ]:
schema

In [ ]:
din_sql = dsl.DIN_SQL(bedrock_model_id=model_id)

쿼리 실행을 준비하기 위해 Athena에 연결합니다.

In [ ]:
din_sql.athena_connect(catalog_name=ATHENA_CATALOG_NAME, 
               db_name=DB_NAME, 
               s3_prefix=ATHENA_RESULTS_S3_LOCATION)

이제 질문의 어떤 단어와도 일치하는 테이블 이름이 있는 테이블 메타데이터를 `schema` 객체에 추가하여, 놓칠 수 있는 명백한 일치 항목을 포착합니다.

In [ ]:
list_tables = din_sql.find_tables(DB_NAME)
list_words = query.split(" ")

intersection = reduce(lambda acc, x: acc + [x] if x in list_words and x not in acc else acc,
                      list_tables, [])
for table in intersection:
    if table in schema:
        print("exists")
    else:
        schema_name = din_sql.get_schema(DB_NAME, table)
        schema[table] = schema_name

이제 `schema` 객체에 무엇이 들어있는지 살펴보겠습니다.

In [ ]:
schema

스키마 정보가 사용할 준비가 되었으므로, 이제 품질 있는 결과를 얻을 수 있는 프롬프트를 작성할 준비가 되었습니다. Claude 프롬프팅 모범 사례를 사용하는 다음 프롬프트를 살펴보고, 스키마가 지시사항에 어떻게 통합되는지 확인해보세요.

In [ ]:
prompt_template = PromptTemplate.from_template(
    """<Instructions>
            <database_schema></database_schema> 태그 안의 데이터베이스 스키마를 읽고 다음을 수행하세요:
            이 스키마는 테이블 이름과 파이프로 구분된 스키마의 json 목록을 포함합니다:
            1. 질문에 답하기 위해 문법적으로 올바른 awsathena 쿼리를 생성하세요.
            2. 특정 테이블의 모든 컬럼을 쿼리하지 마세요. 질문과 관련된 몇 개의 컬럼만 요청하세요.
            3. 스키마 설명에서 볼 수 있는 컬럼 이름만 사용하도록 주의하세요.
            4. 존재하지 않는 컬럼을 쿼리하지 않도록 주의하세요.
            5. 어떤 컬럼이 어떤 테이블에 속하는지 주의하세요.
            6. 필요할 때 테이블 이름으로 컬럼 이름을 한정하세요. 다음 형식을 사용해야 하며, 각각 한 줄씩 작성하세요:
            7. SQL 쿼리를 <sql></sql> 태그 안에 반환하세요.
        </Instructions>

        <database_schema>{schema}</database_schema>

        <examples>
        <question>"사용자가 몇 명인가요?"</question>
        <sql>SELECT SUM(users) FROM customers</sql>

        <question>"모바일 사용자가 몇 명인가요?"</question>
        <sql>SELECT SUM(users) FROM customer WHERE source_medium='Mobile'</sql>
        </examples>

        <question>{input_question}</question>
        """)
prompt = prompt_template.format(schema=schema, input_question=query)
print(prompt)

전체 프롬프트가 준비되었으므로, Claude에게 제출하여 어떤 결과를 얻을 수 있는지 확인해보겠습니다.

In [ ]:
answer = run_bedrock(prompt=prompt)
print(answer)
sql = u.extract_tag(answer, "sql")[0]

In [ ]:
print(sql)

생성한 이 쿼리로 데이터를 조회해보겠습니다.

In [ ]:
results = din_sql.query(sql)
results